In [1]:
import cv2
import numpy as np
import random
from PIL import Image
from IPython.display import display
import os
from multiprocessing import Pool

In [6]:
def create_dataset(name, save_name, save_segment_name):
    width=1700
    height=1700
    cv_background_color=(255,255,0)
    background_color = (0,255,255)
    background_segment = (0,0,0)
    final_size=500

    img = cv2.imread(name, cv2.IMREAD_COLOR)
    segment = np.full((img.shape[0], img.shape[1], 3), [0,128,0], dtype = "uint8")
    img_height, img_width, img_channel = img.shape

    padding_width = random.randrange(width-img_width)
    padding_height = random.randrange(height-img_height)

    noise=np.full((width, height, 3), cv_background_color, dtype = "uint8")
    segment_noise = np.full((noise.shape[0], noise.shape[1], 3), background_segment, dtype = "uint8")

    noise[padding_height:padding_height+img_height, 
           padding_width:padding_width+img_width,:] = img
    segment_noise[padding_height:padding_height+img_height, 
           padding_width:padding_width+img_width,:] = segment
    
    # You may need to convert the color.
    img_rgb = cv2.cvtColor(noise, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(img_rgb)
    segment_pil = Image.fromarray(segment_noise)
    
    angle = random.randrange(359)
    rotate_img = img_pil.rotate(angle, expand=True, fillcolor=background_color)
    rotate_segment = segment_pil.rotate(angle, expand=True, fillcolor=background_segment)
    resize_img = rotate_img.resize((500,500))
    resize_segment = rotate_segment.resize((500,500))

    np_img = np.array(resize_img)
    np_segment = np.array(resize_segment)

    padding_size = random.randrange(100, 400)
    img_height, img_width, img_channel = np_img.shape

    padding_width = random.randrange(padding_size)
    padding_height = random.randrange(padding_size)

    padding_img=np.full((img_width+padding_size, img_width+padding_size, 3), background_color, dtype = "uint8")
    padding_segment = np.full((img_width+padding_size, img_width+padding_size, 3), background_segment, dtype = "uint8")

    # Resize to final img
    padding_img[padding_height:padding_height+img_height, 
           padding_width:padding_width+img_width,:] = np_img
    padding_segment[padding_height:padding_height+img_height, 
           padding_width:padding_width+img_width,:] = np_segment

    pil_img = Image.fromarray(padding_img, mode='RGB')
    pil_segment = Image.fromarray(padding_segment, mode='RGB')
    resize_img = pil_img.resize((500,500))
    resize_segment = pil_segment.resize((500,500))
    final_img = np.array(resize_img)
    final_segment = np.array(resize_segment)
    
    final_img[(final_img == np.array(background_color)).all(axis=2)] = (np.random.rand(3) * 255).astype(int)
#     for i in range(final_img.shape[0]):    # for every pixel:
#         for j in range(final_img.shape[1]):
#             if (final_img[i][j] == np.array(background_color)).all():
#                 final_img[i][j] = 
#             else: 
#                 final_img[0][1] = final_img[0][1]

    final_pil = Image.fromarray(final_img, mode='RGB')
    final_pil.save(save_name)
    final_segment_pil = Image.fromarray(final_segment, mode='RGB')
    final_segment_pil.save(save_segment_name)

In [7]:
images = []
count = 1
for x in os.listdir("./unlabelled"):
    for i in range(3):
        if x.endswith(".jpeg"):
            # Prints only text file present in My Folder
            name = f"./unlabelled/{x}"
            save_name= f"./dataset/images/image_{count}{i}.bmp"
            save_segment_name = f"./dataset/targets/image_{count}{i}_GT.bmp"
            create_dataset(name,save_name,save_segment_name)
        else:
            print(x)
    count +=1
print(count)

KeyboardInterrupt: 

In [ ]:
import random
images = []
count = 1
for x in os.listdir("./dataset/images"):
    images.append(x)
    count += 1
random.shuffle(images)
train_size = int(count*0.6)
test_size = int(count*0.4*0.5)

with open("./dataset/train.txt", 'w') as output:
    for row in images[:train_size]:
        output.write(str(row) + '\n')

with open("./dataset/test.txt", 'w') as output:
    for row in images[train_size:train_size+test_size]:
        output.write(str(row) + '\n')

with open("./dataset/val.txt", 'w') as output:
    for row in images[train_size+test_size:]:
        output.write(str(row) + '\n')
# print(count, len(images[:train_size]), 
#       len(images[train_size:train_size+test_size]),
#       len(images[train_size+test_size:]))